In [1]:
import pandas as pd
import numpy as np

In [2]:
#загрузим датасет
ds = pd.read_csv("SMSSpamCollection", delimiter="\t", header=None)

In [3]:
#посмотрим, что там
ds.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [46]:
#разделим отдельно на предложения и классы
texts=ds[1]
y=[int(x=='spam') for x in ds[0]]

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
#получим вектор признаков по тексту
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import make_scorer, f1_score

In [50]:
f1_scorer = make_scorer(f1_score)
logreg = LogisticRegression()

In [51]:
ans1 = cross_val_score(logreg, X, y, scoring=f1_scorer, cv=10)

In [52]:
print "Ответ 1: среднее значение f1-меры =",np.mean(ans1)

Ответ 1: среднее значение f1-меры = 0.932640298361


In [53]:
logreg.fit_transform(X, y)

C:\Users\Anton\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


<5572x1965 sparse matrix of type '<type 'numpy.int64'>'
	with 56676 stored elements in Compressed Sparse Column format>

In [54]:
test_text = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time",
"Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"]

In [55]:
X_pred = vectorizer.transform(test_text)
ans2 = logreg.predict(X_pred)

In [58]:
print "Результат для сообщений из задания:"
print ans2

Результат для сообщений из задания:
[1 1 0 0 0]


In [60]:
params = [(2,2), (3,3),(1,3)]
for k in params:
    vectorizer = CountVectorizer(ngram_range=k)
    X = vectorizer.fit_transform(texts)
    logreg = LogisticRegression()
    ansk = cross_val_score(logreg, X, y, scoring=f1_scorer, cv=10)
    print "Для n-грамм: ", k, "среднее значение f1-меры = %.2f" % (np.mean(ansk))

Для n-грамм:  (2, 2) среднее значение f1-меры = 0.82
Для n-грамм:  (3, 3) среднее значение f1-меры = 0.73
Для n-грамм:  (1, 3) среднее значение f1-меры = 0.93


In [61]:
from sklearn.naive_bayes import MultinomialNB
params = [(2,2), (3,3),(1,3)]
for k in params:
    vectorizer = CountVectorizer(ngram_range=k)
    X = vectorizer.fit_transform(texts)
    logreg = MultinomialNB()
    ansk = cross_val_score(logreg, X, y, scoring=f1_scorer, cv=10)
    print "Для n-грамм: ", k, "среднее значение f1-меры = %.2f" % (np.mean(ansk))

Для n-грамм:  (2, 2) среднее значение f1-меры = 0.65
Для n-грамм:  (3, 3) среднее значение f1-меры = 0.38
Для n-грамм:  (1, 3) среднее значение f1-меры = 0.89


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
tfidf = TfidfVectorizer(ngram_range=(1,1))
X_tfidf = tfidf.fit_transform(texts)
logreg = LogisticRegression()
ans3 = cross_val_score(logreg, X_tfidf, y, scoring=f1_scorer, cv=10)
print "Для tf*idf признаков среднее значение f1-меры = %.2f" % (np.mean(ans3))

Для tf*idf признаков среднее значение f1-меры = 0.85


Можно сделать вывод, что с признаками tf*idf качество ухадшается, сравнивая результат с аналогичным классификатором и признаками CountVectorizera

Из данного задания можно сделать вывод, что не всегда то, что первое приходит в голову, то есть наиболее простое и очевидное решение, работает хуже, чем последующие решения.
Еще один очевидный вывод: логистическая регрессия работает лучше наивного байеса=)
Также можно сделать вывод о том, что для достижения приемлимого качества в задаче поиска спама в тексте не нужно придумывать хитрые и сложные алгоритмы.